# Azure Cognitive Search Vector Search Code Sample with Azure OpenAI
This code demonstrates how to use Azure Cognitive Search with OpenAI and Azure Python SDK
## Prerequisites
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents --pre`.

In [ ]:
! pip install azure-search-documents --pre
! pip install python-dotenv
! pip install tenacity

## Import required libraries and environment variables

In [ ]:
# Import required libraries  
import os  
import json  
import openai  
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector
from langchain.embeddings import OpenAIEmbeddings
import pandas as pd
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,
    HnswVectorSearchAlgorithmConfiguration
)  
  
# Configure environment variables  
service_endpoint = "https://gptkb-odk2jh5qispeo.search.windows.net"
index_name = "wanda-2022-index"
key = "UZIFuytF5cAeQBg1z5oKmAOS5YD3h1SgzY8bbwsJHKAzSeDAWTMd"
openai.api_type = "azure"  
openai.api_key = "a65f52d60c744eb9b141d9939cd4c4b6"
openai.api_base = "https://openaidemo-hu.openai.azure.com/"
openai.api_version = "2023-05-15"

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://openaidemo-hu.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "a65f52d60c744eb9b141d9939cd4c4b6"

credential = AzureKeyCredential(key)


## Create embeddings
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1, openai_api_version="2023-05-15")


In [ ]:

index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)

## Perform a vector similarity search

通过纯粹的向量搜索，天然的支持多语言的内容

In [ ]:
# Pure Vector Search multi-lingual (e.g 'tools for software development' in Dutch)  
query = "结算收入"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  

vector = Vector(value=embeddings.embed_query(query), k=3, fields="contentVector")

  
results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    select=["title", "content", "url"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"url: {result['url']}\n")  


## Perform a Hybrid Search

通过混合搜索（认知搜素的bing搜索 + 向量搜索）

In [ ]:
# Hybrid Search
query = "结算收入"  
  
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  

vector = Vector(value=embeddings.embed_query(query), k=3, fields="contentVector")
  
results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    select=["title", "content", "url"],
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"url: {result['url']}\n")  


## Perform a Semantic Hybrid Search

通过混合搜索（认知搜素的语义搜索 + 向量搜索）

In [ ]:
# Semantic Hybrid Search
query = "结算收入"

search_client = SearchClient(
    service_endpoint, index_name, AzureKeyCredential(key))

vector = Vector(value=embeddings.embed_query(query), k=3, fields="contentVector")

results = search_client.search(
    search_text=query,
    vectors=[vector],
    select=["title", "content", "url"],
    query_type="semantic", query_language="zh-CN", semantic_configuration_name='default', query_caption="extractive", query_answer="extractive",
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Content: {result['content']}")
    print(f"url: {result['url']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")
